In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv

import paramiko

# sshtunnel.py에서  paramiko_key_types의 항목 중 'dsa': paramiko.DSSKey 주석 처리 (참조 제거)
from sshtunnel import SSHTunnelForwarder
import pymysql

import matplotlib.pyplot as plt

from datetime import datetime
import holidays

import json

current_date = datetime.now().strftime('%Y-%m-%d')

load_dotenv()

SSH_HOST = os.getenv("SSH_HOST")
SSH_PORT = int(os.getenv("SSH_PORT", 22))
SSH_USER = os.getenv("SSH_USER")
SSH_KEY_PATH = os.getenv("SSH_KEY_PATH")

DB_HOST = os.getenv("DB_HOST")
DB_PORT = int(os.getenv("DB_PORT", 3306))
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_ORDER_SERVICE = os.getenv("DB_ORDER_SERVICE")

def query_db(query, params=None):
    # SSH 키 로드
    ssh_key = SSH_KEY_PATH
    
    with SSHTunnelForwarder(
        (SSH_HOST, SSH_PORT),
        ssh_username=SSH_USER,
        ssh_pkey=ssh_key,  # 로드된 키 객체 또는 파일 경로
        remote_bind_address=(DB_HOST, DB_PORT)
    ) as tunnel:
        with pymysql.connect(
            host='127.0.0.1',
            port=tunnel.local_bind_port,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_ORDER_SERVICE,
            charset='utf8mb4'
        ) as conn:    
            try:
                df = pd.read_sql(query, conn, params=params)
                if df.empty and len(df.columns) > 0:
                    with conn.cursor() as cursor:
                        cursor.execute(query, params)
                        rows = cursor.fetchall()
                        if rows:
                            df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
            except Exception as e:
                print(f"[ERROR] 데이터 조회 중 오류: {e}")  # f-string 수정
                raise
    return df     

c:\Users\USER\OneDrive\Desktop\Company\lunchlab\pilot\data_analytics\venv\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\USER\OneDrive\Desktop\Company\lunchlab\pilot\data_analytics\venv\Lib\site-packages\paramiko\transport.py:253: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "class": algorithms.TripleDES,


In [ ]:
from data.database import DatabaseConnection

query = "SELECT * FROM order_service.orders"

df = DatabaseConnection.execute_query(query)

df.head()
result = query_db(query)


2025-11-20 16:40:58.236 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-11-20 16:41:13,305| ERROR   | Exception (client): Error reading SSH protocol banner
2025-11-20 16:41:13,311| ERROR   | Traceback (most recent call last):
2025-11-20 16:41:13,312| ERROR   |   File "c:\Users\USER\OneDrive\Desktop\Company\lunchlab\pilot\data_analytics\venv\Lib\site-packages\paramiko\transport.py", line 2271, in _check_banner
2025-11-20 16:41:13,312| ERROR   |     buf = self.packetizer.readline(timeout)
2025-11-20 16:41:13,313| ERROR   |   File "c:\Users\USER\OneDrive\Desktop\Company\lunchlab\pilot\data_analytics\venv\Lib\site-packages\paramiko\packet.py", line 380, in readline
2025-11-20 16:41:13,313| ERROR   |     buf += self._read_timeout(timeout)
2025-11-20 16:41:13,313| ERROR   |            ~~~~~~~~~~~~~~~~~~^^^^^^^^^
2025-11-20 16:41:13,313| ERROR   |   File "c:\Users\USER\OneDrive\Desktop\Company\lunchlab\pilot\data_analytics\venv\Lib\site

BaseSSHTunnelForwarderError: Could not establish session to SSH gateway